<a href="https://colab.research.google.com/github/tomonari-masada/course2023-sml/blob/main/12_decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tree-based methodsによる住宅価格の予測

* 前に使ったCalifornia housing datasetを使う。

 * scikit-learnからロードできるバージョンは、前処理が済んだキレイなデータなので、ここでは使わない。

* データの取得や前処理の一部は、
[Aurélien Géron. Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, 2nd Edition.](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/) の2章と同じ。

* 今回は、以下の解説を参考にします。
 * https://scikit-learn.org/stable/auto_examples/impute/plot_missing_values.html
* 特に、`sklearn.pipeline.make_pipeline`を用いることで、コードを簡素化します。

In [ ]:
import numpy as np
from scipy import stats, special
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

%config InlineBackend.figure_format = "retina"

## 1) データを取得

In [ ]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
  os.makedirs(housing_path, exist_ok=True)
  tgz_path = os.path.join(housing_path, "housing.tgz")
  urllib.request.urlretrieve(housing_url, tgz_path)
  housing_tgz = tarfile.open(tgz_path)
  housing_tgz.extractall(path=housing_path)
  housing_tgz.close()

In [ ]:
fetch_housing_data()

In [ ]:
def load_housing_data(housing_path=HOUSING_PATH):
  csv_path = os.path.join(housing_path, "housing.csv")
  return pd.read_csv(csv_path)

（ここより上の詳細はフォローしなくてもいいいです。）

In [ ]:
housing = load_housing_data()
housing.head()

## 2) データを概観しつつ前処理

In [ ]:
housing.info()

* 数値データではない列が一つだけある



In [ ]:
housing["ocean_proximity"].value_counts()

* 今回は、この数値データではない列を残す
 * pandasのget_dummiesを使う



In [ ]:
housing_num = pd.get_dummies(housing, columns=["ocean_proximity"])

In [ ]:
housing_num.head()

In [ ]:
housing_num.info()

* 説明変数と目的変数を分ける



In [ ]:
X = housing_num.drop("median_house_value", axis=1)
y = housing_num["median_house_value"]

## 3) 評価実験のための準備

* 今回は交差検証をおこなうので、テストデータだけを切り分けておく。
 * 交差検証には`sklearn.model_selection.cross_val_score`を使う。

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.info()

* 前の授業と同じ分割なので、やはりテストセットに欠測箇所がある。
 * 今回は、後で、訓練データの中央値で埋めることにする。

In [ ]:
X_test.info()

## 4) 前処理

* どのような前処理をおこなうかは、お任せします。

In [ ]:
X_train.hist(bins=50, figsize=(15,12));

## 5) 決定木をチューニング

* 木の深さ（領域を何分割するか）をチューニングする。

* cross_val_scoreで使える評価尺度一覧をチェックする。
 * いずれも、「値が大きいほど良い」という評価尺度になっている。
 * 今回はRMSEで評価したいが、これは「値が小さいほど良い」という評価尺度である。
 * そこで、一覧の中にある'neg_root_mean_squared_error'に、マイナスをつけたものを、使う。
 * これで前の結果と比較できるようになる。



In [ ]:
sorted(metrics.SCORERS.keys())

* 交差検証を、欠損値を埋めつつ実行するために、ヘルパ関数を定義しておく。

In [ ]:
def get_scores(imputer, regressor, X_missing, y_missing, cv=10):
  estimator = make_pipeline(imputer, regressor)
  impute_scores = cross_val_score(
      estimator,
      X_missing, y_missing,
      scoring="neg_root_mean_squared_error",
      cv=cv,
      )
  return impute_scores

* 10-fold cross validationを実施
 * 木の深さは3から20まで変える。
 * 評価尺度はRMSE。



In [ ]:
best_depth = 0
best_score = 0.0
for i in range(3, 16):
  reg = tree.DecisionTreeRegressor(max_depth=i, random_state=123)
  imp = SimpleImputer(missing_values=np.nan, strategy="median")
  scores = get_scores(imp, reg, X_train, y_train, cv=10)
  temp_score = - scores.mean()
  if best_depth == 0 or best_score > temp_score:
    best_depth = i
    best_score = temp_score
  print(f"depth {i}: {temp_score:.2f}")
print('-'*64)
print(f"best depth {best_depth}: {best_score:.2f}")

* 最適な木の深さを使って訓練データ全体で学習をやり直し、テストデータで評価。



In [ ]:
reg = tree.DecisionTreeRegressor(max_depth=best_depth, random_state=123)
imp = SimpleImputer(missing_values=np.nan, strategy="median")
reg.fit(imp.fit_transform(X_train), y_train)
y_test_pred = reg.predict(imp.transform(X_test))
rmse = mean_squared_error(y_test, y_test_pred, squared=False)
print(f"test RMSE: {rmse:.2f}")

* これが今回のベースラインです。
* この値を改善することを試みてください。

# 課題

* RMSEによって評価される予測性能を、良くして下さい
* test setとそれ以外の部分の分割は、変えないでください
 * test set以外の部分をどう使うかは、自由です
 * 交差検証の方法は何でもよいです。
* 決定木ベースの手法やその系統の手法なら、何を使ってもいいです。
* test setでのRMSEによる評価は、最後に一回おこなうだけです

## A) XGBoostを使ってみる



In [ ]:
import xgboost as xgb

* 10-fold cross validationを実施
 * 実行時間はsklearnのDecisionTreeRegressorよりも長め。


In [ ]:
for i in range(6, 14):
  reg = xgb.XGBRegressor(objective ="reg:squarederror", max_depth=i, random_state=123)
  imp = SimpleImputer(missing_values=np.nan, strategy="median")
  scores = get_scores(imp, reg, X_train, y_train, cv=10)
  print(f"depth {i}: {- scores.mean():.2f}")

* 最適な木の深さを使って訓練データ全体で学習をやり直し、テストデータで評価。



In [ ]:
# 各自、実践してください。

## B) CatBoostを使ってみる

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
for i in range(6, 14):
  reg = CatBoostRegressor(iterations=200, depth=i, random_seed=123, logging_level="Silent")
  imp = SimpleImputer(missing_values=np.nan, strategy="median")
  scores = get_scores(imp, reg, X_train, y_train, cv=10)
  print(f"depth {i}: {- scores.mean():.2f}")

* 最適な木の深さを使って訓練データ全体で学習をやり直し、テストデータで評価。

In [ ]:
# 各自、実践してください。


* 他に、決定木に類するモデル、gradient boosting系のモデルを見つけて、色々試してみてください。